In [1]:
import pandas as pd
import numpy as np


data = pd.read_pickle('chrom_real_new.pkl')
data.set_index(['gene', 'intron'], inplace=True)
data.sort_index(inplace=True)
data.head()

chrom  lmm-pval  lmm-rank-pval       pos  qep-pval  \
gene            intron                                                       
ENSG00000000419 5          20  0.757149       0.615790  49508683  0.599888   
                5          20  0.149643       0.489927  49509175  0.523147   
                5          20  0.466112       0.302470  49509184  0.240761   
                5          20  0.364057       0.273795  49511295  0.235351   
                5          20  0.466112       0.302470  49511352  0.240761   

                                 snp_id  
gene            intron                   
ENSG00000000419 5       snp_20_49508683  
                5       snp_20_49509175  
                5       snp_20_49509184  
                5       snp_20_49511295  
                5       snp_20_49511352

# Bonferroni correction

In [2]:
ntests = len(data.index.unique())
print("Number of tests: %d" % ntests)

# Bonferroni within gene-intron
def pval_agg(x):
    return np.clip(np.min(x) * len(x), 0, 1)

def method_agg(x):
    return x[0]
    
def gene_intron_wise(df):
    return data.groupby(level=[0, 1]).agg({
        'lmm-pval': pval_agg,
        'lmm-rank-pval': pval_agg,
        'qep-pval': pval_agg
    })

data = gene_intron_wise(data)

Number of tests: 66551


In [3]:
data.head()

lmm-pval  qep-pval  lmm-rank-pval
gene            intron                                   
ENSG00000000419 5       1.000000  1.000000       1.000000
                6       1.000000  1.000000       0.405983
                7       0.820634  1.000000       1.000000
                8       1.000000  1.000000       1.000000
ENSG00000000457 1       0.007733  0.057588       0.065261

In [4]:
def method_column(df):
    newdata = pd.DataFrame(columns=('pval', 'method'))

    def append_df(data, df, name):
        pval = df['%s-pval' % name]
        return pd.concat([data, pd.DataFrame({'pval': pval, 'method': name})])
        return data

    newdata = append_df(newdata, df, 'lmm')
    newdata = append_df(newdata, df, 'lmm-rank')
    newdata = append_df(newdata, df, 'qep')
    return newdata

In [5]:
def set_index(data):
    data = method_column(data)
    data.reset_index(inplace=True)
    data['gene'] = data['index'].apply(lambda x: x[0])
    data['intron'] = data['index'].apply(lambda x: int(x[1]))
    del data['index']
    data.set_index(['gene', 'intron'], inplace=True)
    data.sort_index(inplace=True)
    return data

In [6]:
data = set_index(data)

In [7]:
from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [8]:
data.head()

method      pval
gene            intron                    
ENSG00000000419 5            lmm  1.000000
                5       lmm-rank  1.000000
                5            qep  1.000000
                6            lmm  1.000000
                6       lmm-rank  0.405983

In [9]:
def plot_curve(data, perc=False, exclude=False, lthr=-5, rthr=-0.30103):
    
    nhits = {'lmm-rank':[], 'qep':[]}
    if not exclude:
        nhits['lmm'] = []
    
    pts = np.logspace(lthr, rthr)
    for pt in pts:

        if perc:
            grouped = data.groupby(data['method']).agg({
                'pval': lambda x: 100*sum(x*len(x)<=pt)/float(len(x))
            })
        else:
            grouped = data.groupby(data['method']).agg({
                'pval': lambda x: sum(x*len(x)<=pt)
            })


        nhits['lmm-rank'].append(grouped.loc['lmm-rank', 'pval'])
        if not exclude:
            nhits['lmm'].append(grouped.loc['lmm', 'pval'])
        nhits['qep'].append(grouped.loc['qep', 'pval'])

    p = figure(title="Power", tools=['save,reset'])
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'P-value threshold'
    if perc:
        p.yaxis.axis_label = 'Percentage of hits'
    else:
        p.yaxis.axis_label = 'Number of hits'
    p.line(pts, nhits['lmm-rank'], color='green', legend='lmm-rank')
    if not exclude:
        p.line(pts, nhits['lmm'], color='blue', legend='lmm')
    p.line(pts, nhits['qep'], color='red', legend='qep')
    p.legend.location = "bottom_right"
    return p

In [10]:
lthr = -5
rthr = -1
p1 = plot_curve(data, False, lthr=lthr, rthr=rthr)
p2 = plot_curve(data, False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2]], plot_width=400, plot_height=400))

In [11]:
from horta_exp.introns.fetch_data import get_intron_events
introns = get_intron_events()
introns.head()

assay  nsuc  ntri
gene            intron                                  
ENSG00000000419 5       HG00096.1.M_111124_6    37   347
                5       HG00097.7.M_120219_2    68   451
                5       HG00099.1.M_120209_6    31   327
                5       HG00099.5.M_120131_3    34   289
                5       HG00100.2.M_111215_8    35   281

In [12]:
def plot_data(data, title='title'):
    print(title)
    lthr = -5
    rthr = -1
    p1 = plot_curve(data, True, lthr=lthr, rthr=rthr)
    p2 = plot_curve(data, True, True, lthr=lthr, rthr=rthr)
    show(gridplot([[p1,p2]], plot_width=400, plot_height=400, title=title))

In [13]:
g = introns.groupby(level=[0, 1])
g = g.agg({
    'ntri': {
        'max': 'max',
        'min': 'min',
    },
    'nsuc': {
        'max': 'max',
        'min': 'min',
    }
})
g.head()

nsuc      ntri     
                        max min   max  min
gene            intron                    
ENSG00000000419 5       148   5   951   29
                6         9   0   675    8
                7        50   0   863   28
                8       571   4  1243  104
ENSG00000000457 1        83   0  1309   73

In [14]:
for th in range(50, 501, 50):
    d = data.loc[g['ntri']['max'] < th]
    plot_data(d, title='Max #ntrials: %d; Number of gene-introns: %d' % (th, len(d.index.unique())))

th = 1000
d = data.loc[g['ntri']['max'] < th]
plot_data(d, title='Max #ntrials: %d; Number of gene-introns: %d' % (th, len(d.index.unique())))

th = 10000
d = data.loc[g['ntri']['max'] < th]
plot_data(d, title='Max #ntrials: %d; Number of gene-introns: %d' % (th, len(d.index.unique())))

Max #ntrials: 50; Number of gene-introns: 43


Max #ntrials: 100; Number of gene-introns: 1367


Max #ntrials: 150; Number of gene-introns: 4371


Max #ntrials: 200; Number of gene-introns: 8371


Max #ntrials: 250; Number of gene-introns: 12669


...

Max #ntrials: 400; Number of gene-introns: 25419


Max #ntrials: 450; Number of gene-introns: 29147


Max #ntrials: 500; Number of gene-introns: 32712


Max #ntrials: 1000; Number of gene-introns: 55612


Max #ntrials: 10000; Number of gene-introns: 66550
